# PageRank
#### Author : Alex Fender

In this notebook, we will show how to use multi-GPU features in cuGraph to run the PageRank on a 300GB dataset on a DGX2.

Please be aware that your system may be different, and you may need to modify the code or install packages to run the below examples. If you think you have found a bug or an error, please file an issue in [cuGraph](https://github.com/rapidsai/cugraph/issues)

This notebook was run using RAPIDS 0.10 and CUDA 10.0. 

## Introduction
Pagerank is measure of the relative importance of a vertex based on the relative importance of its neighbors.  PageRank was invented by Google Inc. and is (was) used to rank its search results. PageRank uses the connectivity information of a graph to rank the importance of each vertex. See [Wikipedia](https://en.wikipedia.org/wiki/PageRank) for more details on the algorithm.

CuGraph's multi-GPU features leverage Dask. RAPIDS has other projects based on Dask such as dask-cudf and dask-cuda. These products will also be used in this example. Check out [RAPIDS.ai](https://rapids.ai/) to learn more about these technologies.

---

To compute the Pagerank with cuGraph we use:<br>

```python
cugraph.dask.pagerank.pagerank(edge_list, alpha=0.85, max_iter=30)
```
Parameters

*  *edge_list* : `dask_cudf.DataFrame`<br>
Contain the connectivity information as an edge list. Source 'src' and destination 'dst' columns must be of type 'int32'. Edge weights are not used for this algorithm. Indices must be in the range [0, V-1], where V is the global number of vertices. The input edge list should be provided in dask-cudf DataFrame with one partition per GPU.
*  *alpha* : `float`<br>
The damping factor alpha represents the probability to follow an outgoing edge, standard value is 0.85. Thus, 1.0-alpha is the probability to “teleport” to a random vertex. Alpha should be greater than 0.0 and strictly lower than 1.0.
* *max_iter* : `int`<br>
The maximum number of iterations before an answer is returned. If this value is lower or equal to 0 cuGraph will use the default value, which is 30. In this notebook, we will use 20 to compare against published results.<br>

Returns

* *PageRank* : `dask_cudf.DataFrame`<br>
Dask GPU DataFrame containing two columns of size V: the vertex identifiers and the corresponding PageRank values.

## Data
We will be analyzing 400 million nodes and 16 billion links from an artificial HiBench dataset (Zipfian distribution). The CSV edge list file is 300GB and split into 32 partitions.

---

We recommend that you download and decompress the data on a ***local*** directory on the DGX-2 machine. Notice that on clusters, most user sessions are NFS-based which would be a huge performance bottleneck if the dataset is stored there. Your system may be different, and you may want to talk to your cluster administrator to identify the best storage option.

---

The fastest way to obtain the dataset is to run :
```
wget https://rapidsai-data.s3.us-east-2.amazonaws.com/cugraph/benchmark/hibench/HiBench_300GB.tar.gz
tar -xzf HiBench_300GB.tar.gz
```


## Multi-GPU PageRank with cuGraph
### Basic setup

In [ ]:
# Let's check out our hardware setup
!nvidia-smi

# GPUs should be connected with NVlink
!nvidia-smi nvlink --status

# List available devices (we need all 16 GPUs)
# If empty, all GPUs are available by default
!echo Available GPUs: $CUDA_VISIBLE_DEVICES

In [ ]:
# Import needed libraries. We recommend using cugraph_dev env via conda
import time
from dask.distributed import Client, wait
import dask_cudf
from dask_cuda import LocalCUDACluster
import cugraph.dask.pagerank as dcg

### Setup multi-GPU and Dask

Before we get started, we need to setup a Dask local cluster of workers to execute our work and a client to coordinate and schedule work for that cluster. As we see below, we can initiate a `cluster` and `client` using only 2 lines of code.

In [ ]:
# By default, Dask will use the current directory to store temporary information
cluster = LocalCUDACluster(threads_per_worker=1)
# If your working directory is on a NFS, you will want to change the line above to
# specify a local directory instead :
# cluster = LocalCUDACluster(local_dir="/some_local_path_on the current_machine/",threads_per_worker=1)

client = Client(cluster)

### Read the data from disk
cuGraph depends on dask-cudf for data loading and the initial DataFrame creation. The CSV data file contains an edge list, which represents the connection of a vertex to another. The source to destination pairs is what is known as Coordinate Format (COO). In this test case, the data is just two columns. 

CuGraph has a special `read_split_csv` for large datasets which cannot be read directly by dask-cudf due to memory requirements. This function takes large input split into smaller files (number of input files > number of gpus), reads two or more csv per gpu/worker and concatenates them into a single dataframe. Additional parameters (delimiter, names and dtype) can be specified for reading the csv file.

In [ ]:
# File path
# *** edit this ***
input_data_path = r"/datasets/"

# Files
input_files = ['file-00000.csv','file-00001.csv','file-00002.csv','file-00003.csv',
             'file-00004.csv','file-00005.csv','file-00006.csv','file-00007.csv',
             'file-00008.csv','file-00009.csv','file-00010.csv','file-00011.csv',
             'file-00012.csv','file-00013.csv','file-00014.csv','file-00015.csv',
             'file-00016.csv','file-00017.csv','file-00018.csv','file-00019.csv',
             'file-00020.csv','file-00021.csv','file-00022.csv','file-00023.csv',
             'file-00024.csv','file-00025.csv','file-00026.csv','file-00027.csv',
             'file-00028.csv','file-00029.csv','file-00030.csv','file-00031.csv']

# Concatenate file paths
files = [input_data_path+f for f in input_files]

# Start timer
t_start = time.time()

# Special Multi-GPU CSV reader for splited files
e_list = dcg.read_split_csv(files)

# Wait for the lazy reader
tmp = wait(client.compute(e_list.to_delayed()))

# Print time
print(time.time()-t_start, "s")

### Call the Multi-GPU PageRank algorithm


In [ ]:
# Start timer
t_start = time.time()

# Get the pagerank scores
pr_ddf = dcg.pagerank(e_list, max_iter=20)

# Print time
print(time.time()-t_start, "s")

It was that easy! cuGraph should only take a few seconds per iteration to run on this 300GB input with 16 Tesla V100 GPUs.<br>

### Close the multi-GPU environment

In [ ]:
client.close()
cluster.close()

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___